In [ ]:
import os
import time
from dotenv import load_dotenv
import azure.cognitiveservices.speech as speechsdk
from azure.ai.translation.text import TextTranslationClient
from azure.core.credentials import AzureKeyCredential
from pydub import AudioSegment
from datetime import datetime

# 환경변수 로드
load_dotenv()

# Azure 설정
speech_api_key = os.getenv("SPEECH_API_KEY")
region = os.getenv("SPEECH_REGION")
translator_api_key = os.getenv("TRANSLATOR_API_KEY")
translator_region = os.getenv("TRANSLATOR_REGION")
translator_endpoint = os.getenv("TRANSLATOR_ENDPOINT")

# ==================== 언어 설정 ====================
# 여기서 입력 언어와 출력 언어를 설정하세요
INPUT_LANGUAGE = "한국어"      # STT 입력 언어
OUTPUT_LANGUAGE = "영어"       # 번역 출력 + TTS 출력 언어

# 언어 매핑 테이블
LANGUAGE_MAPPING = {
    "한국어": ("ko-KR", "ko", "ko-KR-SunHiNeural"),
    "영어": ("en-US", "en", "en-US-AvaMultilingualNeural"),
    "중국어(간체)": ("zh-CN", "zh-Hans", "zh-CN-XiaoxiaoNeural"),
    "일본어": ("ja-JP", "ja", "ja-JP-NanamiNeural"),
    "독일어": ("de-DE", "de", "de-DE-KatjaNeural"),
    "프랑스어": ("fr-FR", "fr", "fr-FR-DeniseNeural"),
    "스페인어": ("es-ES", "es", "es-ES-ElviraNeural"),
    "이탈리아어": ("it-IT", "it", "it-IT-IsabellaNeural"),
    "러시아어": ("ru-RU", "ru", "ru-RU-DariyaNeural"),
    "포르투갈어": ("pt-BR", "pt", "pt-BR-FranciscaNeural"),
}

INPUT_STT_CODE, INPUT_TRANSLATE_CODE, INPUT_TTS_VOICE = LANGUAGE_MAPPING[INPUT_LANGUAGE]
OUTPUT_STT_CODE, OUTPUT_TRANSLATE_CODE, OUTPUT_TTS_VOICE = LANGUAGE_MAPPING[OUTPUT_LANGUAGE]

print(f"🎤 입력 언어: {INPUT_LANGUAGE} ({INPUT_STT_CODE})")
print(f"🔄 출력 언어: {OUTPUT_LANGUAGE} ({OUTPUT_TRANSLATE_CODE})")
print("=" * 50)

# STT 설정
speech_config = speechsdk.SpeechConfig(subscription=speech_api_key, region=region)
speech_config.speech_recognition_language = INPUT_STT_CODE

# FFmpeg 경로 설정
AudioSegment.converter = r"C:\\ffmpeg\\bin\\ffmpeg.exe"
AudioSegment.ffprobe = r"C:\\ffmpeg\\bin\\ffprobe.exe"

# 번역기 설정
translator_credential = AzureKeyCredential(translator_api_key)
translator = TextTranslationClient(
    credential=translator_credential,
    endpoint=translator_endpoint,
    region=translator_region
)

# TTS 설정
tts_config = speechsdk.SpeechConfig(subscription=speech_api_key, region=region)
tts_config.speech_synthesis_voice_name = OUTPUT_TTS_VOICE

# ==================== 번역 함수 ====================
def translate_text(text, target_language=OUTPUT_TRANSLATE_CODE):
    """텍스트 번역"""
    try:
        response = translator.translate(
            body=[{"text": text}],
            to_language=[target_language]
        )
        
        translation_result = response[0]
        translated_text = translation_result.translations[0].text
        
        detected_language = None
        if hasattr(translation_result, 'detected_language') and translation_result.detected_language:
            detected_language = translation_result.detected_language.language
        
        return {
            "original_text": text,
            "translated_text": translated_text,
            "detected_language": detected_language
        }
    except Exception as e:
        print(f"❌ 번역 오류: {e}")
        return {"original_text": text, "translated_text": None, "error": str(e)}

# ==================== TTS 함수 ====================
def speak_text(text, output_method="speaker"):
    """TTS 음성 출력"""
    if not text.strip():
        return None
    
    try:
        if output_method == "speaker":
            audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
        else:  # file
            now = datetime.now().strftime("%y%m%d_%H%M%S")
            filename = f"tts_output_{now}.wav"
            audio_config = speechsdk.audio.AudioOutputConfig(filename=filename)
        
        synthesizer = speechsdk.SpeechSynthesizer(tts_config, audio_config)
        result = synthesizer.speak_text_async(text).get()
        
        if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
            if output_method == "speaker":
                print(f"🔊 [{OUTPUT_LANGUAGE}] TTS 출력 완료")
                return True
            else:
                print(f"💾 [{OUTPUT_LANGUAGE}] 파일 저장: {filename}")
                return filename
        else:
            print(f"❌ TTS 실패: {result.cancellation_details.reason}")
            return False
    except Exception as e:
        print(f"❌ TTS 오류: {e}")
        return None

# ==================== STT → 번역 → TTS 파이프라인 ====================
def process_stt_results(all_results, auto_translate=True, auto_tts=True, tts_method="speaker"):
    """STT 결과를 번역하고 TTS로 출력"""
    if not all_results:
        print("❌ 인식된 텍스트가 없습니다.")
        return
    
    # STT 결과 합치기
    original_text = " ".join(all_results)
    print(f"\n📝 원본 텍스트 ({INPUT_LANGUAGE}): {original_text}")
    
    if auto_translate:
        # 번역
        result = translate_text(original_text)
        if result.get("translated_text"):
            translated = result["translated_text"]
            detected = result.get("detected_language", "unknown")
            print(f"🔄 번역 결과 ({OUTPUT_LANGUAGE}): {translated}")
            print(f"   감지된 언어: {detected}")
            
            if auto_tts:
                # TTS 출력
                speak_text(translated, tts_method)
        else:
            print("❌ 번역 실패")

# ==================== 메인 실행부 ====================
def main():
    print("🎯 Azure STT → 번역 → TTS 통합 시스템")
    print("=" * 50)
    print("1. 마이크 실시간 처리")
    print("2. 오디오 파일 처리")
    print("0. 종료")
    
    while True:
        choice = input("\n선택하세요 > ").strip()
        
        if choice == "1":
            # 마이크 입력 처리
            print(f"🎤 {INPUT_LANGUAGE} 마이크 입력 시작 (10초 후 자동 종료)")
            
            # 마이크 입력 설정
            audio_input = speechsdk.audio.AudioConfig(use_default_microphone=True)
            speech_recognizer = speechsdk.SpeechRecognizer(speech_config, audio_input)
            
            all_results = []
            
            def recognized_handler(evt):
                if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
                    print(f"✅ 인식: {evt.result.text}")
                    all_results.append(evt.result.text)
            
            def stop_handler(evt):
                print("🛑 인식 세션 종료")
                speech_recognizer.stop_continuous_recognition()
            
            speech_recognizer.recognized.connect(recognized_handler)
            speech_recognizer.session_stopped.connect(stop_handler)
            speech_recognizer.canceled.connect(stop_handler)
            
            # 연속 인식 시작
            speech_recognizer.start_continuous_recognition()
            
            try:
                time.sleep(10)  # 10초 대기
            except KeyboardInterrupt:
                print("\n사용자가 중단했습니다.")
            
            speech_recognizer.stop_continuous_recognition()
            
            # 결과 처리
            process_stt_results(all_results, auto_translate=True, auto_tts=True, tts_method="speaker")
            
        elif choice == "2":
            # 파일 입력 처리  
            # file_path = input("오디오 파일 경로 입력 > ").strip()
            file_path = "audio1.wav"
            
            if not os.path.exists(file_path):
                print("❌ 파일이 존재하지 않습니다.")
                continue
            
            print(f"📁 파일 처리 중: {file_path}")
            
            # 파일 형식 확인 및 변환
            direct_process_exts = [".wav", ".pcm", ".wave", ".flac"]
            file_ext = os.path.splitext(file_path)[1].lower()
            
            if file_ext in direct_process_exts:
                print(f"{file_ext} 형식, 변환 없이 인식 처리")
                audio_filepath = file_path
            else:
                print(f"{file_ext} 형식, WAV PCM으로 변환 수행")
                audio_converted = "audio_converted.wav"
                sound = AudioSegment.from_file(file_path)
                sound = sound.set_channels(1).set_frame_rate(16000)
                sound.export(audio_converted, format="wav", parameters=["-ac", "1", "-ar", "16000"])
                audio_filepath = audio_converted
            
            # STT 처리
            audio_input = speechsdk.audio.AudioConfig(filename=audio_filepath)
            speech_recognizer = speechsdk.SpeechRecognizer(speech_config, audio_input)
            
            all_results = []
            
            def recognized_handler(evt):
                if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
                    print(f"✅ 인식: {evt.result.text}")
                    all_results.append(evt.result.text)
            
            def stop_handler(evt):
                print("🛑 인식 세션 종료")
                speech_recognizer.stop_continuous_recognition()
            
            speech_recognizer.recognized.connect(recognized_handler)
            speech_recognizer.session_stopped.connect(stop_handler)
            speech_recognizer.canceled.connect(stop_handler)
            
            # 연속 인식 시작
            speech_recognizer.start_continuous_recognition()
            
            # 오디오 길이 계산 및 대기
            if os.path.exists(audio_filepath):
                audio = AudioSegment.from_file(audio_filepath)
                duration_sec = len(audio) / 1000.0
                print(f"⏱️ 인식할 오디오 길이: {duration_sec:.2f}초")
                time.sleep(duration_sec + 2)
            
            speech_recognizer.stop_continuous_recognition()
            
            # 결과 처리
            process_stt_results(all_results, auto_translate=True, auto_tts=True, tts_method="speaker")
            
        elif choice == "0":
            print("👋 프로그램을 종료합니다.")
            break
            
        else:
            print("❌ 잘못된 선택입니다.")

if __name__ == "__main__":
    main()


python-dotenv could not parse statement starting at line 21
python-dotenv could not parse statement starting at line 23


🎤 입력 언어: 한국어 (ko-KR)
🔄 출력 언어: 영어 (en)
🎯 Azure STT → 번역 → TTS 통합 시스템
1. 실시간 처리 (마이크)
2. 파일 처리
0. 종료
📁 파일 처리: audio1.wav
📝 원본 텍스트: 데이터.
🔄 번역 결과: Data.
🔊 [영어] TTS 출력: Data.
✅ TTS 출력 완료
프로그램을 종료합니다.
